In [80]:
'''
    @author  jasonYu
    @date    2017/6/3
    @version created
    @email   yuquanjie13@gmail.com
'''
import numpy as np
import cv2
from PIL import Image, ImageFont, ImageDraw
import os
import sys
import string
import re
sys.path.append('/home/yuquanjie/Documents/deep-direct-regression/tools')
from point_check import point_in_polygon

'''
1. generate intput data
    1.1 input image resizing, from (2400,3200) to (320,320)
    1.2 (TODO) zero-center by mean pixel??
    1.3 proprecess image
2. generate output data
    2.1 generate classification output data
    2.2 (TODO) generate regression output data
'''
def get_train_data(all_imgs):
    visulise = False
    while True:
        for img_data in all_imgs:      
            print img_data['imagePath']
            # image file wheater corresponding to text fle
            annot = img_data['imagePath']
            strinfo = re.compile('image/')
            annot = strinfo.sub('text/',annot)
            strinfo = re.compile('jpg')
            annot = strinfo.sub('txt',annot)
            
            if os.path.isfile(img_data['imagePath']) and os.path.isfile(annot):
                img = cv2.imread(img_data['imagePath'])
                width = img.shape[0] #2400
                height = img.shape[1] #3200

                ## 1)generate input data
                ### 1.1)input image, from (2400,3200) to (320,320)
                img_320 = cv2.resize(img,(320,320),interpolation=cv2.INTER_CUBIC)

                ## 2)generate output data
                ### 2.1)generate classification output data
                divi_x = float(height) / 80.0
                divi_y = float(width) / 80.0
                #y_class_lable = -1 * np.ones((80,80)).astype(np.float32)
                y_class_lable = -1 * np.ones((80,80))
                for ix in xrange(y_class_lable.shape[0]):
                    for jy in xrange(y_class_lable.shape[1]):
                        for polygon in img_data['boxCoord']:
                            x1 = string.atof(polygon[0]) / divi_x
                            x2 = string.atof(polygon[2]) / divi_x
                            x3 = string.atof(polygon[4]) / divi_x
                            x4 = string.atof(polygon[6]) / divi_x

                            y1 = string.atof(polygon[1]) / divi_y
                            y2 = string.atof(polygon[3]) / divi_y
                            y3 = string.atof(polygon[5]) / divi_y
                            y4 = string.atof(polygon[7]) / divi_y

                            polygon = [(x1,y1), (x2,y2), (x3,y3), (x4,y4)]
                            if point_in_polygon(ix,jy,polygon):
                                y_class_lable[ix][jy] = 1
                            #else:
                            #    y_class_lable[ix][jy] = 0   

                if visulise:
                    if img_data['imagePath'] == '/home/yuquanjie/Documents/icdar2017rctw_train_v1.2/train/part1/image/image_100.jpg' :
                        img = cv2.imread(img_data['imagePath'])
                        img_80 = cv2.resize(img,(80,80),interpolation=cv2.INTER_CUBIC)
                        img_draw = Image.fromarray(cv2.cvtColor(img_80,cv2.COLOR_BGR2RGB))
                        draw = ImageDraw.Draw(img_draw)
                        for coord in img_data['boxCoord']:
                            print 'detail'
                            print float(coord[0]) / divi_x, float(coord[1]) / divi_y
                            print float(coord[2]) / divi_x, float(coord[3]) / divi_y
                            print float(coord[4]) / divi_x, float(coord[5]) / divi_y
                            print float(coord[6]) / divi_x, float(coord[7]) / divi_y
                            print 'detail'
                            draw.polygon([(float(coord[0]) / divi_x, float(coord[1]) / divi_y), 
                                          (float(coord[2]) / divi_x, float(coord[3]) / divi_y),
                                          (float(coord[4]) / divi_x, float(coord[5]) / divi_y), 
                                          (float(coord[6]) / divi_x, float(coord[7]) / divi_y)],
                                         outline = "red",fill="blue")
                        img_draw = np.array(img_draw)
                        img_draw = cv2.cvtColor(img_draw, cv2.COLOR_RGB2BGR)
                        one_locs = np.where(y_class_lable > 0)
                        print one_locs
                        print len(one_locs[0])
                        print img_data['imagePath']
                        print img_data['boxNum']
                        cv2.imshow('img',img_draw)
                        cv2.waitKey(0)        
                
                img_320 = np.expand_dims(img_320,axis = 0)
                one_locs = np.where(y_class_lable > 0)
                y_class_lable = np.expand_dims(y_class_lable,axis = 0)
                y_class_lable = np.expand_dims(y_class_lable,axis = 3)
                #yield np.copy(img_320), np.copy(y_class_lable), img_data
                
                ### 2.2)(TODO) generate regression output data
                #y_regr_lable = np.zeros((80,80,8)).astype(np.float32)                   
                y_regr_lable = np.zeros((80,80,8))
                for i in xrange(len(one_locs[0])):
                    # get quadrilateral vertex 4 corrdinates
                    for polygon in img_data['boxCoord']:
                        x1 = string.atof(polygon[0]) / divi_x
                        x2 = string.atof(polygon[2]) / divi_x
                        x3 = string.atof(polygon[4]) / divi_x
                        x4 = string.atof(polygon[6]) / divi_x

                        y1 = string.atof(polygon[1]) / divi_y
                        y2 = string.atof(polygon[3]) / divi_y
                        y3 = string.atof(polygon[5]) / divi_y
                        y4 = string.atof(polygon[7]) / divi_y

                        poly = [(x1,y1), (x2,y2), (x3,y3), (x4,y4)]
                        ix = one_locs[0][i]
                        jy = one_locs[1][i]
                        if point_in_polygon(ix,jy,poly):
                            left_top_x = poly[0][0]
                            left_top_y = poly[0][1]
                            righ_top_x = poly[1][0]
                            righ_top_y = poly[1][1]

                            righ_dow_x = poly[2][0]
                            righ_dow_y = poly[2][1] 
                            left_dow_x = poly[3][0]
                            left_dow_y = poly[3][1]

                            y_regr_lable[ix][jy][0] = left_top_x * 4 - ix * 4
                            y_regr_lable[ix][jy][1] = left_top_y * 4 - jy * 4
                            y_regr_lable[ix][jy][2] = righ_top_x * 4 - ix * 4
                            y_regr_lable[ix][jy][3] = righ_top_y * 4 - jy * 4

                            y_regr_lable[ix][jy][4] = righ_dow_x * 4 - ix * 4
                            y_regr_lable[ix][jy][5] = righ_dow_y * 4 - jy * 4
                            y_regr_lable[ix][jy][6] = left_dow_x * 4 - ix * 4
                            y_regr_lable[ix][jy][7] = left_dow_y * 4 - jy * 4
                if visulise and img_data['imagePath'] == '/home/yuquanjie/Documents/icdar2017rctw_train_v1.2/train/part1/image/image_100.jpg' :
                    print y_regr_lable[59][75]
                    print y_regr_lable[59][76]
                    print y_regr_lable[59][77]
                y_regr_lable = np.expand_dims(y_regr_lable,axis = 0)
                yield np.copy(img_320), np.copy(y_class_lable), np.copy(y_regr_lable), img_data
            else:
                continue

In [81]:
from keras import backend as K
import tensorflow as tf
HUBER_DELTA = 1.0


def smoothL1(y_true, y_pred):
    #print y_pred.shape
    import tensorflow as tf
    #1. slice
    #conTmp = tf.slice(y_true, [0, 0, 0, 8],[1, 80, 80, 1])
    #2. concatenate
    #tmp = tf.expand_dims(y_true[:, :, :, 8], 3)  page 27 helped by hl
    tmp = tf.expand_dims(y_true[:, :, :, 8], 3)
    #print tmp
    
    
    y_true = tf.concat([y_true, tmp], 3)
    y_true = tf.concat([y_true, tmp], 3)
    y_true = tf.concat([y_true, tmp], 3)
    
    y_true = tf.concat([y_true, tmp], 3)
    y_true = tf.concat([y_true, tmp], 3)
    y_true = tf.concat([y_true, tmp], 3)
    y_true = tf.concat([y_true, tmp], 3)
    
    #print y_true
    x = K.abs(y_true[:, :, :, 0:8] - y_pred)
    if K._BACKEND == 'tensorflow':
        import tensorflow as tf
        x = tf.where(tf.greater(HUBER_DELTA, x), 
                     0.5 * x ** 2, 
                     x - 0.5)
        x = tf.where(tf.greater(y_true[:, :, :,8:16], 0),
                     y_true[:, :, :,8:16],
                     0 * y_true[:, :, :,8:16]) * x
        #return  K.sum(x)
        return  K.mean(x, axis = -1)
#def mergeLoss():

In [82]:
from keras.layers import Input, Add, Dense, Activation, Flatten, Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers.convolutional import Conv2DTranspose
from keras.layers.merge import add
from keras.layers.core import Lambda
from keras import backend as K

def multi_task(input_tensor=None, trainable=False):
    img_input = input_tensor
    bn_axis = 3
    
    #conv_1
    conv1_1 = Convolution2D(32, (5, 5), strides=(1,1), padding='same',
                            activation='relu', name='conv1_1')(img_input)
    pool1 = MaxPooling2D((2,2), strides=(2,2), name='pool1')(conv1_1)
           
    #conv_2
    conv2_1 = Convolution2D(64, (3, 3), strides=(1,1), padding='same',
                            activation='relu', name='conv2_1')(pool1)
    conv2_2 = Convolution2D(64, (3, 3), strides=(1,1), padding='same', 
                            activation='relu', name='conv2_2')(conv2_1)
    pool2 = MaxPooling2D((2,2), strides=(2,2), name='pool2')(conv2_2)
    
    #conv_3    
    conv3_1 = Convolution2D(128, (3, 3), strides=(1,1), padding='same', 
                            activation='relu', name='conv3_1')(pool2)
    conv3_2 = Convolution2D(128, (3, 3), strides=(1,1), padding='same', 
                            activation='relu', name='conv3_2')(conv3_1)
    pool3 = MaxPooling2D((2,2), strides=(2,2), name='pool3')(conv3_2)
    pool3_for_fuse = Convolution2D(128, (1, 1), strides=(1,1), padding='same',
                                   activation='relu', name='pool3_for_fuse')(pool3)
    
    #conv_4    
    conv4_1 = Convolution2D(256, (3, 3), strides=(1,1), padding='same', 
                            activation='relu', name='conv4_1')(pool3)
    conv4_2 = Convolution2D(256, (3, 3), strides=(1,1), padding='same', 
                            activation='relu', name='conv4_2')(conv4_1)
    pool4 = MaxPooling2D((2,2), strides=(2,2), name='pool4')(conv4_2)
    pool4_for_fuse = Convolution2D(128, (1, 1), strides=(1,1), padding='same',
                                   activation='relu', name='pool4_for_fuse')(pool4)
    
    #conv_5    
    conv5_1 = Convolution2D(512, (3, 3), strides=(1,1), padding='same', 
                            activation='relu', name='conv5_1')(pool4)
    conv5_2 = Convolution2D(512, (3, 3), strides=(1,1), padding='same', 
                            activation='relu', name='conv5_2')(conv5_1)
    pool5 = MaxPooling2D((2,2), strides=(2,2), name='pool5')(conv5_2)
    pool5_for_fuse = Convolution2D(128, (1, 1), strides=(1,1), padding='same',
                                   activation='relu', name='pool5_for_fuse')(pool5)
    
    #conv_6    
    conv6_1 = Convolution2D(512, (3, 3), strides=(1,1), padding='same', 
                            activation='relu', name='conv6_1')(pool5)
    conv6_2 = Convolution2D(512, (3, 3), strides=(1,1), padding='same', 
                            activation='relu', name='conv6_2')(conv6_1)
    pool6 = MaxPooling2D((2,2), strides=(2,2), name='pool6')(conv6_2)
    
    #
    conv7_1 = Convolution2D(128, (1, 1), strides=(1,1), padding='same', 
                            activation='relu', name='conv7_1')(pool6)
    
    upscore2 = Conv2DTranspose(filters=128, kernel_size=(2, 2),
                              strides=(2, 2), padding='valid', use_bias=False,
                             name='upscore2')(conv7_1)
    
    fuse_pool5 = add([upscore2, pool5_for_fuse])
    upscore4 = Conv2DTranspose(filters=128, kernel_size=(2, 2),
                              strides=(2, 2), padding='valid', use_bias=False,
                             name='upscore4')(fuse_pool5)
    fuse_pool4 = add([upscore4, pool4_for_fuse])
       
    upscore8 = Conv2DTranspose(filters=128, kernel_size=(2, 2),
                              strides=(2, 2), padding='valid', use_bias=False,
                             name='upscore8')(fuse_pool4)
    fuse_pool3 = add([upscore8, pool3_for_fuse])
    
    upscore16 = Conv2DTranspose(filters=128, kernel_size=(2, 2),
                              strides=(2, 2), padding='valid', use_bias=False,
                             name='upscore16')(fuse_pool3)
    ##########################################################################
    ####### shared layer
    ##########################################################################
    x_clas = Convolution2D(1, (1, 1), strides=(1,1), padding='same', name='out_class')(upscore16)
    x = Convolution2D(128, (1, 1), strides=(1,1), padding='same', activation='relu')(upscore16)
    x = Convolution2D(8, (1, 1), strides=(1,1), padding='same', activation='sigmoid')(x)
    x_regr = Lambda(lambda t: 800 * t - 400)(x)
    #x_merge = Merge(x_)
    return [x_clas, x_regr, x]

In [83]:
import numpy as np
import cv2
import time
from PIL import Image, ImageFont, ImageDraw
from keras.layers import Input
from keras import optimizers
from keras.callbacks import ModelCheckpoint
import h5py
from keras.models import Model
import sys
sys.path.append('/home/yuquanjie/Documents/deep-direct-regression/tools')
from point_check import point_in_polygon
from get_data import get_raw_data
import os
gpu_id = '1'
os.environ['CUDA_VISIBLE_DEVICES']=str(gpu_id)

all_imgs, numFileTxt = get_raw_data('/home/yuquanjie/Documents/icdar2017rctw_train_v1.2/train/part1')
data_gen_train = get_train_data(all_imgs)

img_input = Input((320,320,3))
# Create model
multi = multi_task(img_input,trainable=True)
#multask_model = Model(img_input, multi[0:2])
multask_model = Model(img_input, multi[1])
#print multask_model.summary()
sgd = optimizers.SGD(lr=0.01, decay=4e-4, momentum=0.9)
# Compile model
#multask_model.compile(loss=['hinge',smoothL1], optimizer=sgd)
multask_model.compile(loss=[smoothL1], optimizer=sgd)

if False:
    X_train, Y_train_cls, Y_train_regr, Z  = data_gen_train.next()
    Y_train_merge = np.concatenate([Y_train_regr, Y_train_cls], axis = 3)
    for i in range (100):
        X_train_iter, Y_train_cls_iter, Y_train_regr_iter, Z  = data_gen_train.next()

        X_train = np.concatenate([X_train, X_train_iter], axis = 0)
        Y_train_cls = np.concatenate([Y_train_cls, Y_train_cls_iter], axis = 0)

        Y_train_merge_iter = np.concatenate([Y_train_regr_iter, Y_train_cls_iter], axis = 3)
        Y_train_merge = np.concatenate([Y_train_merge, Y_train_merge_iter], axis = 0)


X = X_train
Y = [Y_train_cls, Y_train_merge]
Y_1 = Y[0]
Y_2 = Y[1]

read_data = False
if read_data:
    # wirte data
    file = h5py.File('trian_dataset_test.h5','w')
    file.create_dataset('X_train',data = X_train)
    file.create_dataset('Y_train_cls',data = Y_train_cls)
    file.create_dataset('Y_train_merge',data = Y_train_merge)
    file.close()   
    # read data
    print "reading data from trian_dataset_test.h5 "
    file = h5py.File('trian_dataset_test.h5','r')
    X = file['X_train'][:]
    Y_1 = file['Y_train_cls'][:]
    Y_2 = file['Y_train_merge'][:]
    file.close()

print X.shape
print Y_1.shape
print Y_2.shape
Y = [Y_1, Y_2]
# checkpoint 
filepath = "/home/yuquanjie/Documents/deep-direct-regression/model/loss-decrease-{epoch:02d}-{loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor = 'loss', verbose = 1, 
                             save_best_only = True, mode='min')
callbacks_list = [checkpoint]
# Fit the model
loss_class = multask_model.fit(X, Y_2, batch_size=5, epochs=500,callbacks = callbacks_list, verbose=1)
#loss_class = multask_model.fit(X, Y, batch_size=5, epochs=500, verbose=1)

Parsing annotation files
(101, 320, 320, 3)
(101, 80, 80, 1)
(101, 80, 80, 9)
Epoch 1/500
101/101 [==============================] - 13s - loss: 38.5697    
Epoch 2/500
101/101 [==============================] - 4s - loss: 39.1490     
Epoch 3/500
101/101 [==============================] - 4s - loss: 39.1810     
Epoch 4/500
101/101 [==============================] - 4s - loss: 39.1853     
Epoch 5/500
101/101 [==============================] - 4s - loss: 39.1853     
Epoch 6/500
101/101 [==============================] - 4s - loss: 39.1853     
Epoch 7/500
101/101 [==============================] - 4s - loss: 39.1853     
Epoch 8/500
101/101 [==============================] - 4s - loss: 39.1853     
Epoch 9/500
101/101 [==============================] - 4s - loss: 39.1853     
Epoch 10/500
101/101 [==============================] - 4s - loss: 39.1853     
Epoch 11/500
101/101 [==============================] - 4s - loss: 39.1853     
Epoch 12/500
101/101 [=============================

KeyboardInterrupt: 

In [ ]:
import numpy as np
import cv2
import time
from PIL import Image, ImageFont, ImageDraw
from keras.layers import Input
from keras import optimizers
from keras.callbacks import ModelCheckpoint
import h5py
from keras.models import Model, load_model
import sys
sys.path.append('/home/yuquanjie/Documents/deep-direct-regression/tools')
from point_check import point_in_polygon
from get_data import get_raw_data
import os
gpu_id = '1'
os.environ['CUDA_VISIBLE_DEVICES']=str(gpu_id)


all_imgs, numFileTxt = get_raw_data('/home/yuquanjie/Documents/icdar2017rctw_train_v1.2/train/part1')
data_gen_train = get_train_data(all_imgs)
#######
if True:
    X_train, Y_train_cls, Y_train_regr, Z  = data_gen_train.next()
    Y_train_merge = np.concatenate([Y_train_regr, Y_train_cls], axis = 3)
    for i in range (0):
        X_train_iter, Y_train_cls_iter, Y_train_regr_iter, Z  = data_gen_train.next()

        X_train = np.concatenate([X_train, X_train_iter], axis = 0)
        Y_train_cls = np.concatenate([Y_train_cls, Y_train_cls_iter], axis = 0)

        Y_train_merge_iter = np.concatenate([Y_train_regr_iter, Y_train_cls_iter], axis = 3)
        Y_train_merge = np.concatenate([Y_train_merge, Y_train_merge_iter], axis = 0)


X = X_train
print X.shape
Y = [Y_train_cls, Y_train_merge]
#######


img_input = Input((320,320,3))
# Create model
multi = multi_task(img_input,trainable=True)
multask_model = Model(img_input, multi[0:2])
#print multask_model.summary()
sgd = optimizers.SGD(lr=0.01, decay=4e-4, momentum=0.9)
# Compile model
multask_model.compile(loss=['hinge',smoothL1], optimizer=sgd)

final_model = load_model('model/loss-decrease-999-4.07.hdf5', custom_objects={'smoothL1': smoothL1})
P_result = final_model.predict_on_batch(X)
print P_result[0].shape
print P_result[1].shape
print np.where(P_result[0][0, :, :, 0] > 0)
np.set_printoptions(threshold=1e6)
print P_result[1][0, :, :, 0]

In [5]:
import tensorflow as tf
a = tf.constant([[[1, 1, 1, 1], 
                 [1, 1, 1, 1], 
                 [0, 0, 0, 0]],
                [[0, 0, 0, 0], 
                 [0, 0, 0, 0], 
                 [0, 0, 0, 0]]
                ])
#num_pos = tf.reduce_sum(a)
#print num_pos
#with tf.Session() as sess:
#    print(sess.run(a))
#    print(sess.run(los))

ImportError: Traceback (most recent call last):
  File "/usr/local/anaconda2/lib/python2.7/site-packages/tensorflow/python/pywrap_tensorflow.py", line 41, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/usr/local/anaconda2/lib/python2.7/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/usr/local/anaconda2/lib/python2.7/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
ImportError: libcusolver.so.7.5: cannot open shared object file: No such file or directory


Failed to load the native TensorFlow runtime.

See https://www.tensorflow.org/install/install_sources#common_installation_problems

for some common reasons and solutions.  Include the entire stack trace
above this error message when asking for help.